# __ETL__ _(Extract, Transform, Load)_

## Introducción

Este notebook se enfoca en el proceso de **ETL** utilizando datos extraídos de las plataformas Yelp y Google Maps. Este proceso implica una _extracccion,transformación y carga_ de los datos con el objetivo de prepararlos para análisis posteriores. Este paso es crucial en cualquier proyecto de ciencia de datos para garantizar la calidad y utilidad de los datos.

## Configuraciones Globales e Importaciones

En esta sección, se instalan e importan todas las librerías y/o módulos necesarios para el proceso ETL (Extract, Transform, Load) y se establecen configuraciones globales de ser requerido. Se utilizan las siguientes librerías y herramientas:

In [1]:
import warnings
warnings.filterwarnings("ignore") # Se utiliza para gestionar las advertencias y mantener el código limpio.

In [2]:
import os # Proporciona funciones para interactuar con el sistema operativo.
import pandas as pd # Una librería de análisis de datos.
import json # Se utiliza para trabajar con datos en formato JSON.

# YELP

**Dataset:** CHECKIN.JSON

DECLARACIÓN DE LA RUTA DE LOS DATA SET

In [6]:
# Ruta local completa del archivo checkin.json
ruta_local_checkin = r"C:/Users/Usuario/Desktop/Proyecto Final/Yelp/checkin.json"

In [7]:
checkin_yelp = []

with open(ruta_local_checkin, "r", encoding='utf-8') as filejson:
    # Procesa cada línea del archivo como un objeto JSON
    for linea in filejson:
        # Intenta cargar la línea como un objeto JSON
        objeto = json.loads(linea)
        checkin_yelp.append(objeto)

# Convierte la lista de diccionarios a un DataFrame de Pandas
checkin_yelp = pd.DataFrame(checkin_yelp)

In [9]:
#MUestra las primero 5 filas del dataframe
checkin_yelp.head(5)

,business_id,date
0,---kPU91CF4Lq2-WlRu9Lw,"2020-03-13 21:10:56, 2020-06-02 22:18:06, 2020..."
1,--0iUa4sNDFiZFrAdIWhZQ,"2010-09-13 21:43:09, 2011-05-04 23:08:15, 2011..."
2,--30_8IhuyMHbSOcNWd6DQ,"2013-06-14 23:29:17, 2014-08-13 23:20:22"
3,--7PUidqRWpRSpXebiyxTg,"2011-02-15 17:12:00, 2011-07-28 02:46:10, 2012..."
4,--7jw19RH9JKXgFohspgQw,"2014-04-21 20:42:11, 2014-04-28 21:04:46, 2014..."


In [10]:
#Muesta la informacion del dataframe
checkin_yelp.info()
checkin_yelp.shape[0]

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 131930 entries, 0 to 131929
Data columns (total 2 columns):
 #   Column       Non-Null Count   Dtype 
---  ------       --------------   ----- 
 0   business_id  131930 non-null  object
 1   date         131930 non-null  object
dtypes: object(2)
memory usage: 2.0+ MB


131930

In [12]:
# Divide la columna 'date' en una lista de fechas
checkin_yelp['date'] = checkin_yelp['date'].str.split(', ')

In [13]:
# Convierte la lista de fechas en filas separadas
checkin_yelp = checkin_yelp.explode('date').reset_index(drop=True)

In [14]:
# Convierte la columna 'date' al formato datetime
checkin_yelp['date'] = pd.to_datetime(checkin_yelp['date'])

In [15]:
# Ordena el DataFrame por 'business_id' y 'date'
checkin_yelp = checkin_yelp.sort_values(by=['business_id', 'date'])

In [16]:
# Agrega una columna 'checkin_id' con un ID único para cada registro
checkin_yelp['checkin_id'] = checkin_yelp.groupby(level=0).cumcount()

In [17]:
# Reordena las columnas
checkin_yelp = checkin_yelp[['checkin_id', 'business_id', 'date']]

In [18]:
#MUestra las primero 5 filas del dataframe
checkin_yelp.head(5)

,checkin_id,business_id,date
0,0,---kPU91CF4Lq2-WlRu9Lw,2020-03-13 21:10:56
1,0,---kPU91CF4Lq2-WlRu9Lw,2020-06-02 22:18:06
2,0,---kPU91CF4Lq2-WlRu9Lw,2020-07-24 22:42:27
3,0,---kPU91CF4Lq2-WlRu9Lw,2020-10-24 21:36:13
4,0,---kPU91CF4Lq2-WlRu9Lw,2020-12-09 21:23:33


## Carga de nuestro archivo

In [19]:
ruta_nuevo_json = "C:/Users/Usuario/Desktop/Proyecto Final/PF_Google_yelp_Map/Notebook/checkin_limpio.json"
checkin_yelp.to_json(ruta_nuevo_json, orient='records', lines=True)
